In [1]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta

In [2]:
num_users = 1
start_date = "2023-01-01"
end_date = "2024-07-31"
output_file="bank_history.csv"

start_date = datetime.strptime(start_date, "%Y-%m-%d")
end_date = datetime.strptime(end_date, "%Y-%m-%d")

In [3]:
# 사용자 번호 리스트 생성
users_id = [f'user_{i+1}' for i in range(num_users)]

In [4]:
# 사용자별 고정 금액 할당
user_salaries = {user_id: np.random.randint(2500000, 5000000) for user_id in users_id}  # 월급
# loan_repayment = {user_id: np.random.randint(0, 500000) for user_id in users_id}  # 부채상환

# # 상시 입출금 항목 리스트 및 금액 범위
bankhistory_name = {
    '입금:보험금': (10000, 200000),
    '입금:배당금': (1000, 100000),
    '입금:정부지원금': [100000, 200000, 300000],
    '입금:당근거래입금': [5000, 10000, 20000, 30000, 50000]
    # '출금:용돈': (-500000, -50000)
}

In [8]:
# 데이터프레임 초기화
bankhistory_data = {
    'User_ID': [],
    'Date': [],
    'Transaction_Type': [],
    'Amount': [],
    'Fixed' : []
}

# 입출금 내역 생성
for user_id in users_id:
    
    # 초기 설정
    current_date = start_date

    while current_date <= end_date:
        
        # # 해당 월의 랜덤 입출금 내역 생성
        if current_date.day == 1:
            num_transactions = np.random.randint(0, 2)  # 한 달에 0 ~ 2건의 거래 발생
            lastday_of_month = (current_date.replace(month=current_date.month % 12 + 1, day=1) - timedelta(days=1)).day  # 월말일
            transaction_days = random.sample(range(1, lastday_of_month + 1), num_transactions)
            transaction_days.sort()
            
            for date in transaction_days:
                transaction_date = current_date.replace(day=date)  # 랜덤day를 기반으로 datetime형식 저장
                # 발생 항목 및 금액 생성
                transaction_type = random.choice(list(bankhistory_name.keys()))  # 항목 랜덤으로 뽑기
                amount_range = bankhistory_name[transaction_type]  # 거래 유형에 해당하는 금액 범위 가져오기
                if isinstance(amount_range, tuple):  # 금액범위가 튜플이면,
                    amount = np.random.randint(amount_range[0], amount_range[1])  # 범위 내에서 랜덤 생성
                else:  # 금액범위가 리스트면,
                    amount = np.random.choice(amount_range)  # 금액 중에서 랜덤 생성
                
                # 데이터 만들기
                bankhistory_data['User_ID'].append(user_id)
                bankhistory_data['Date'].append(transaction_date.strftime('%Y-%m-%d'))
                bankhistory_data['Transaction_Type'].append(transaction_type)
                bankhistory_data['Amount'].append(amount)
                bankhistory_data['Fixed'].append(False)
        
        # 매월 10일 월급 입금
        if current_date.day == 10:
            bankhistory_data['User_ID'].append(user_id)
            bankhistory_data['Date'].append(current_date.strftime('%Y-%m-%d'))
            bankhistory_data['Transaction_Type'].append('입금:월급')
            bankhistory_data['Amount'].append(user_salaries[user_id])  # 미리 할당된 월급 금액
            bankhistory_data['Fixed'].append(True)
        
        # # 매월 1일 대출금 고정출금
        # if current_date.day == 1:
        #     bankhistory_data['User_ID'].append(user_id)
        #     bankhistory_data['Date'].append(current_date)
        #     bankhistory_data['Transaction_Type'].append('출금:대출상환')
        #     bankhistory_data['Amount'].append(-loan_repayment[user_id])  # 미리 할당된 금액
        #     bankhistory_data['Fixed'] = True
        #     # transaction_data['Amount'].append(-np.random.randint(500000, 2000000))  # 매번랜덤

        current_date += timedelta(days=1)

In [ ]:
# 데이터프레임 생성
bankhistory_df = pd.DataFrame(bankhistory_data)
bankhistory_df

In [22]:
# CSV 파일로 저장
bankhistory_df.to_csv(f'./data/{output_file}', index=False, encoding='cp949')